# Tracing Basics

### Setup

Make sure you set your environment variables, including your OpenAI API key.

In [3]:
from dotenv import load_dotenv
load_dotenv()


True

### Tracing with @traceable

The @traceable decorator is a simple way to log traces from the LangSmith Python SDK. Simply decorate any function with @traceable.

The decorator works by creating a run tree for you each time the function is called and inserting it within the current trace. The function inputs, name, and other information is then streamed to LangSmith. If the function raises an error or if it returns a response, that information is also added to the tree, and updates are patched to LangSmith so you can detect and diagnose sources of errors. This is all done on a background thread to avoid blocking your app's execution.

In [6]:
# TODO: Import traceable
from openai import OpenAI
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

MODEL_PROVIDER = "openai"
MODEL_NAME = "gpt-4o-mini"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()

# TODO: Set up tracing for each function
def retrieve_documents(question: str):
    return retriever.invoke(question)   # NOTE: This is a LangChain vector db retriever, so this .invoke() call will be traced automatically


def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)


def call_openai(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )


def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


Fetching pages: 100%|##########| 197/197 [01:15<00:00,  2.61it/s]


@traceable handles the RunTree lifecycle for you!

In [8]:
question = "How can I trace with the @traceable decorator? What does the traceable decorator mean? What are some usecases for the @traceable decorator"
ai_answer = langsmith_rag(question)
print(ai_answer)

The @traceable decorator in LangSmith allows you to easily log traces from your Python functions with minimal code changes. By decorating a function with @traceable, you can automatically capture and log its execution details, which is useful for monitoring and debugging applications. Use cases include tracking API calls, logging model runs, and analyzing performance in applications that utilize machine learning models.


##### Let's take a look in LangSmith!

### Adding Metadata

LangSmith supports sending arbitrary metadata along with traces.

Metadata is a collection of key-value pairs that can be attached to runs. Metadata can be used to store additional information about a run, such as the version of the application that generated the run, the environment in which the run was generated, or any other information that you want to associate with a run. Similar to tags, you can use metadata to filter runs in the LangSmith UI, and can be used to group runs together for analysis.

In [9]:
from langsmith import traceable

@traceable(
    # TODO: Add Metadata
    # metadata={"vectordb": "sklearn"}
)
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

@traceable(
    # TODO: Add Metadata
    # metadata={"model_name": MODEL_NAME, "model_provider": MODEL_PROVIDER}
)
def call_openai(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    return openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


In [10]:
question = "Why should I add Metadata in the program myself?"
ai_answer = langsmith_rag(question)
print(ai_answer)

Adding metadata allows you to store important information about your examples, such as tags or version info, which can be useful for analyzing experiment results. It also enables you to filter and group examples effectively when using the SDK. This enhances the organization and retrieval of data during your work.


You can also add metadata at runtime!

In [11]:
question = "How do I add metadata at runtime? Is it better to add metadata from before or is it better at runtime?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"runtime_metadata": "foo"}})
print(ai_answer)

You can add metadata to your examples by clicking on an example and then selecting "Edit" in the popover. It's generally better to add metadata before running experiments to ensure consistency and facilitate analysis, but adding it at runtime can be useful for capturing dynamic information. Ultimately, the choice depends on your specific use case and the importance of the metadata for your analysis.


In [12]:
question = "What is meant by tweaking the code?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"Nikhil": "Sharma"}})
print(ai_answer)

Tweaking the code generally refers to making small adjustments or modifications to the existing codebase to improve functionality, fix bugs, or enhance performance. This can involve changing parameters, refining algorithms, or optimizing processes without overhauling the entire system. The goal is often to achieve better results or adapt the code to new requirements.
